# Getting Your Data From Yelp!

In order to make sure you are on track to completing the project, you will complete this workbook first. Below are steps that you need to take in order to make sure you have your data from yelp and are ready to analyze it. Your cohort lead will review this workbook with you the Wednesday before your project is due.    

# Part 1 - Understanding your data and question

You will be pulling data from the Yelp API to complete your analysis. The API, however, provides you with a lot of information that will not be pertinent to your analysis. You will pull data from the API and parse through it to keep only the data that you will need. In order to help you identify that information,look at the API documentation and understand what data the API will provide you. 

Identify which data fields you will want to keep for your analysis. 

https://www.yelp.com/developers/documentation/v3/get_started

# Part 2 - Create ETL pipeline for the business data from the API

## Details

Now that you know what data you need from the API, you want to write code that will execute an API call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*API call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

## Request

### Imports and Setup

In [1]:
import requests
import pandas as pd
import json
import csv
import numpy as np

with open(r'C:\Users\bmcca\.secret\yelp_api.json') as f:
    keys = json.load(f)

client_id = keys['id']
yelp_key = keys['key']

### ƒ: yelp_request

 - Params: search term (eg. "wineries); location; yelp_key variable (from Imports); and changing setting to print details

In [5]:
def yelp_request(term, location, yelp_key, verbose=True):
    '''Adapted from Yelp API Lab: https://github.com/BenJMcCarty/dsc-yelp-api-lab/tree/solution'''
    
    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(yelp_key),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': 50
                }
    response = requests.get(url, headers=headers, params=url_params)
    
    if verbose == True:
        print(response)
        print(type(response.text))
        print(response.text[:1000])
        
    return response.json()

### Sending the request


- Un-comment the next line to run the response

In [6]:
response = yelp_request('winery','Southern California', yelp_key)
response.keys()

<Response [200]>
<class 'str'>
{"businesses": [{"id": "4N2pC-h3Ba6qUQ7cN4xP-g", "alias": "san-antonio-winery-ontario", "name": "San Antonio Winery", "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/__5IBirgkaHoRGdVnUIdng/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/san-antonio-winery-ontario?adjust_creative=eyY_X0k6PMB2bu6EM1dXdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=eyY_X0k6PMB2bu6EM1dXdw", "review_count": 248, "categories": [{"alias": "wineries", "title": "Wineries"}], "rating": 4.0, "coordinates": {"latitude": 34.022224, "longitude": -117.558985}, "transactions": [], "price": "$$", "location": {"address1": "2802 S Milliken Ave", "address2": null, "address3": "", "city": "Ontario", "zip_code": "91761", "country": "US", "state": "CA", "display_address": ["2802 S Milliken Ave", "Ontario, CA 91761"]}, "phone": "+19099473995", "display_phone": "(909) 947-3995", "distance": 26348.268371330654}, {"id": "Q0Cuh5XZY29AdrWpUv8yhw", "alias": 

dict_keys(['businesses', 'total', 'region'])

### Identifying and Exploring Keys

In [7]:
# Identify keys

print(response.keys())

dict_keys(['businesses', 'total', 'region'])


#### Exploring the "Businesses" Key

In [8]:
response['businesses']

[{'id': '4N2pC-h3Ba6qUQ7cN4xP-g',
  'alias': 'san-antonio-winery-ontario',
  'name': 'San Antonio Winery',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/__5IBirgkaHoRGdVnUIdng/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/san-antonio-winery-ontario?adjust_creative=eyY_X0k6PMB2bu6EM1dXdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=eyY_X0k6PMB2bu6EM1dXdw',
  'review_count': 248,
  'categories': [{'alias': 'wineries', 'title': 'Wineries'}],
  'rating': 4.0,
  'coordinates': {'latitude': 34.022224, 'longitude': -117.558985},
  'transactions': [],
  'price': '$$',
  'location': {'address1': '2802 S Milliken Ave',
   'address2': None,
   'address3': '',
   'city': 'Ontario',
   'zip_code': '91761',
   'country': 'US',
   'state': 'CA',
   'display_address': ['2802 S Milliken Ave', 'Ontario, CA 91761']},
  'phone': '+19099473995',
  'display_phone': '(909) 947-3995',
  'distance': 26348.268371330654},
 {'id': 'Q0Cuh5XZY29AdrWpUv8yhw',
  '

In [176]:
# Show first item w/in list of businesses

response['businesses'][0]

{'id': '4N2pC-h3Ba6qUQ7cN4xP-g',
 'alias': 'san-antonio-winery-ontario',
 'name': 'San Antonio Winery',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/__5IBirgkaHoRGdVnUIdng/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/san-antonio-winery-ontario?adjust_creative=eyY_X0k6PMB2bu6EM1dXdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=eyY_X0k6PMB2bu6EM1dXdw',
 'review_count': 248,
 'categories': [{'alias': 'wineries', 'title': 'Wineries'}],
 'rating': 4.0,
 'coordinates': {'latitude': 34.022224, 'longitude': -117.558985},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '2802 S Milliken Ave',
  'address2': None,
  'address3': '',
  'city': 'Ontario',
  'zip_code': '91761',
  'country': 'US',
  'state': 'CA',
  'display_address': ['2802 S Milliken Ave', 'Ontario, CA 91761']},
 'phone': '+19099473995',
 'display_phone': '(909) 947-3995',
 'distance': 26348.268371330654}

In [10]:
response['businesses'][0]['categories'][0]['alias']

'wineries'

In [11]:
response['businesses'][0]['categories'][0]['title']

'Wineries'

#### Exploring the "Total" Key

In [12]:
response['total']

# How many businesses are there in total for my request?

494

#### Exploring the "Region" Key

In [13]:
response['region']

# From which geographical area will my results come?

{'center': {'longitude': -117.784423828125, 'latitude': 34.16815524147703}}

## Parse

### ƒ: parse_data

In [14]:
def parse_data(list_of_data):
    '''Adapted from Tyrell's code'''  

    # Create empty list to store results
    
    parsed_data = []
    
    # Loop through each business in the list of businesses
    # Add specific k:v pairs to a dictionary
    
    for business in list_of_data:
        if 'price' not in business:
            business['price'] = np.nan
            
            # Verify that the "price" key is in the selected business dict
            
        details = {'name': business['name'],
                     'location': ' '.join(business['location']['display_address']),
                     'id': business['id'],
                     'alias': business['categories'][0]['alias'],
                     'title': business['categories'][0]['title'],
                     'rating': business['rating'],
                     'review_count': business['review_count'],
                     'price': business['price'],
                     'latitude': business['coordinates']['latitude'],
                     'longitude': business['coordinates']['longitude']
                    }
        # Add the new dictionary to the previous list
        
        parsed_data.append(details)
    
    # Create a DataFrame from the resulting list
    
    df_parsed_data = pd.DataFrame(parsed_data)

    
    return df_parsed_data

## Updating Requests for Pagination

### ƒ: yelp_request_offset

In [15]:
def yelp_request_offset(term, location, yelp_key, offset=0, verbose=False):
    '''Adapted from Yelp API Lab: https://github.com/BenJMcCarty/dsc-yelp-api-lab/tree/solution'''
    
    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(yelp_key),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': 50,
                    'offset': offset
                        }
    
    response = requests.get(url, headers=headers, params=url_params)
    
    if verbose == True:
        print(response)
        print(type(response.text))
        print(response.text[:1000])
        
    return response.json()

#### Test 1

In [16]:
# test1 = yelp_request_offset('winery', 'San Diego', yelp_key)
# test1

In [17]:
# test1.keys()

In [18]:
# test1['total']

In [19]:
# test1['businesses'][0]

## Edit and Condense

Goal: condense the details down to specific functions (perhaps one function?) to pull, clean, and save data.

While it will take some experimentation to write the functions above, once you get them working it will be best to put them in a `.py` file and then import the functions to use in a script 

# ƒ: GET BUSINESSES (ALL)

In [20]:
def get_full_data(term, location, yelp_key, file_name = 'data/wineries_raw.csv'):
    '''Requests all results from Yelp API; saves as a .csv; and returns a DataFrame.'''

    # Create a .csv to store results
    blank_df = pd.DataFrame()
    blank_df.to_csv(file_name)
    
    # Process first request to Yelp API and calculate number of pages 
    results = yelp_request_offset(term, location, yelp_key, offset=0, 
                                  verbose=False)
    num_pages = results['total']//50+1
    
    # Print out confirmation feedback
    print(f'For {term} and {location}: ')
    print(f"    Total number of results: {results['total']}.")
    print(f'    Total number of pages: {num_pages}.')
    
    # Create offset for further results and create empty list
    offset = 0

    # Retrieves remaining pages
    for num in range(num_pages-1):
        try:
            # Process API request
            results = yelp_request_offset(term, location, yelp_key,
                                          offset=offset, verbose=False)
            
            # From results, take values from "Businesses" key and save
            parsed_results = parse_data(results['businesses'])
          
            # Save resulting DF to .csv from top
            parsed_results.to_csv(file_name, mode='a', index = False)
            
            # Increase offset to move to next "page" of data
            offset += 50
            
        except:
            # If error, print where the error happens
            print(f'Error on page {num}.')
            # Then save the results so far to the .csv
            parsed_results.to_csv(file_name, mode='a', index = False)


    return parsed_results

In [21]:
# # Read saved data from .csv from above

# df2 = pd.read_csv('data/wineries_raw.csv', header = 0)
# df2

# Cleaning Data

# ƒ: SORT_BY_ALIASES

Identifying, Filtering for, and Saving Top 2 Aliases

In [22]:
def sort_by_aliases(raw_data = 'data/wineries_raw.csv'):

    # Read in businesses
    df2 = pd.read_csv(raw_data, header = 0)

    # Create new DF filtering alias and title columns
    df2_alias = df2.loc[:,['alias', 'title']]

    # Identify top 2 aliases 
    df2_alias_count = df2_alias.groupby('alias').count().sort_values(['title'],\
                                                            ascending=False)[:2]

    # Note: initially tried top 3, but it returned distributors, not wineries

    df2_alias_count.reset_index(inplace=True)
    
    print("Top two aliases: ")
    print(df2_alias_count)

    # display them as a list
    aliases_top_2 = df2_alias_count['alias'].tolist()

    # Selecting rows based on condition and saving

    df3 = df2[df2['alias'].isin(aliases_top_2)]

    df3.to_csv('data/wineries_filtered_alias.csv', index = False)
    print("Saved to 'data/wineries_filtered_alias.csv'")
    
    return df3

In [23]:
df4 = sort_by_aliases()
df4

Top two aliases: 
             alias  title
0         wineries     49
1  winetastingroom     33
Saved to 'data/wineries_filtered_alias.csv'


,name,location,id,alias,title,rating,review_count,price,latitude,longitude
1,Bernardo Winery,"13330 Paseo Del Verano Norte San Diego, CA 92128",DknnpiG1p4OoM1maFshzXA,winetastingroom,Wine Tasting Room,4.5,626,$$,33.032800,-117.046460
3,Callaway Vineyard & Winery,"517 4th Ave Ste 101 San Diego, CA 92101",Cn2_bpTngghYW1ej4zreZg,winetastingroom,Wine Tasting Room,5.0,100,$$,32.710751,-117.160918
4,Négociant Winery,"2419 El Cajon Blvd San Diego, CA 92104",Cc1sQWRWgGyMCjzX2mmMQQ,winetastingroom,Wine Tasting Room,4.5,103,$$,32.754880,-117.138280
5,San Pasqual Winery - Seaport Village,"805 W Harbor Dr San Diego, CA 92101",gMW1RvyLu90RSQAY9UrIHw,winetastingroom,Wine Tasting Room,4.5,138,$$,32.708732,-117.168195
6,Domaine Artefact Vineyard & Winery,"15404 Highland Valley Rd Escondido, CA 92025",WqVbxY77Ag96X90LultCUw,wineries,Wineries,5.0,96,$$,33.068170,-117.001600
...,...,...,...,...,...,...,...,...,...,...
140,Roll OutThe Barrell Charity Event by Meritage,"162 S Rancho Santa Fe Rd Encinitas, CA 92024",wyLm9fIoamN-VALcu3nUVg,wineries,Wineries,4.0,1,NaN,33.037121,-117.238654
145,Licores Kentucky,Calle Puerto y 3ra S/N Col. Centro 22000 Tijua...,B7gID-M2EsdpthrTcwTNYA,wineries,Wineries,5.0,1,NaN,32.534236,-117.034976
148,Barrica 9,Av. Revolución 1265 Col. Zona Centro 22000 Tij...,HxTqmzT4G43iAKXrB3pqQg,winetastingroom,Wine Tasting Room,4.5,7,$$,32.530430,-117.036500
165,"RL Liquid Assets, Inc","5909 Sea Lion Pl Ste G Carlsbad, CA 92010",#NAME?,wineries,Wineries,3.0,2,NaN,33.134743,-117.248093


In [24]:
df_saved = pd.read_csv('data/wineries_filtered_alias.csv')
df_saved.keys()

Index(['name', 'location', 'id', 'alias', 'title', 'rating', 'review_count',
       'price', 'latitude', 'longitude'],
      dtype='object')

In [109]:
df4.isna().sum()

name             0
location         0
id               0
alias            0
title            0
rating           0
review_count     0
price           25
latitude         0
longitude        0
dtype: int64

In [115]:
df4['price'].value_counts()

$$     53
$       3
$$$     1
Name: price, dtype: int64

In [117]:
df4.fillna('$$', inplace=True) #NAME?

In [159]:
df5 = df4.where(df4.id == '#NAME?')

df6 = df5[df5.isna() != True]
df6.dropna()

,name,location,id,alias,title,rating,review_count,price,latitude,longitude
77,Hungry Hawk Vineyards & Winery,"3255 Summit Dr Escondido, CA 92025",#NAME?,wineries,Wineries,4.5,118.0,$$,33.093880,-117.030080
165,"RL Liquid Assets, Inc","5909 Sea Lion Pl Ste G Carlsbad, CA 92010",#NAME?,wineries,Wineries,3.0,2.0,$$,33.134743,-117.248093


# Part 3 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews. Above you have a model of the functions you will need to write, and how to pull them together in one script. For this part, you have the process below 

## Getting Business IDs

- In order to pull the reveiws, you will need the business ids. So your first step will be to get all of the business ids from your businesses csv. 

### Open file and slice ID

1. Open data/wineries.csv
2. Slice out the 'name' and 'id' columns for each row

In [25]:
df_saved = pd.read_csv("data/wineries_filtered_alias.csv")
df_saved.reset_index(drop=True, inplace=True)

# Slice out IDs, then save them to a list

df_saved_id = df_saved['id'].to_list()

biz_name = df_saved['name'].to_list()

cleaned_ids = []

for id in df_saved_id:
    if id != '#NAME?':
        cleaned_ids.append(id)
        
cleaned_ids
# biz_name

['DknnpiG1p4OoM1maFshzXA',
 'Cn2_bpTngghYW1ej4zreZg',
 'Cc1sQWRWgGyMCjzX2mmMQQ',
 'gMW1RvyLu90RSQAY9UrIHw',
 'WqVbxY77Ag96X90LultCUw',
 'yZp9FdMH6Dmn98mfNInFHw',
 'ElE6Nj7iz-tNV4ebV6Clew',
 'Ub2bJsi7lIOQ9TyIKdHaJw',
 'sCET1pLdKNNPBQJyjPOkww',
 'fyh566YXm5XJ3Ntv_GLghg',
 '7i8AdZoDySoEdT_uoMMzNA',
 'ALTc2EAwGkWfVL9gpuGdqw',
 'Phqm2QvJE0uEqpxMvdzH_g',
 'NKsyDqmKiNWT4HtkBh6AdA',
 'o0flINbzGa8LDlM9Mtbfwg',
 'Oic9__gID0APdrBIjP4Xjw',
 'YVMiPOAPsEd1fXxNmpjEMA',
 '9qalQETuaZnFDgZrsEB7MA',
 'XH7njeu3FvhxoTqzHTTE5g',
 '1PS0mcUZkJPyG72pk3zwVg',
 'Ye4Pr0ZbC8sGPwpWqR5wvQ',
 'ptA-I2Kk78jydAeJIYs2Cg',
 'f62YFfiriHapBhadXfMquA',
 'BGfXAtLC5sHBWOKlcb17KA',
 'gFPq8yqFRd9caMgkO5HvpQ',
 '97NwIhRGPhv1c4DU9DMpPA',
 'wLEvfL3dKvMMPD17-lpcGw',
 '9YDsJ5OL1hkx_vqwlJhi_Q',
 'VksLDK5jI-O53y0fFoUlQQ',
 '0gGOIJo3MUVpuy8hAepCGQ',
 'ia3GRsrEDexiodnpXe0D6w',
 '5fkdGagUJDuc3axRWchzJQ',
 'D9rtDihupzA39yBh0YcAXg',
 'RdMY32yM5MRIztG9gzuDKw',
 'V3kR-0Nb6ISBQ8ebXZJeVQ',
 '3mk6Tl3irp7x8baYoLRu9w',
 'Wjn9nDudZgJ5okiB8Cbfmg',
 

## Requesting Reviews

- Write a function that takes a business id and makes a call to the API for reviews.


### ƒ: get_reviews

In [54]:
def get_reviews(biz_id, yelp_key, verbose=False):
    '''Adapted from Yelp API Lab: https://github.com/BenJMcCarty/dsc-yelp-api-lab/tree/solution'''
    
    url = 'https://api.yelp.com/v3/businesses/'+ biz_id + '/reviews'

    headers = {
            'Authorization': 'Bearer {}'.format(yelp_key),
        }

    response = requests.get(url, headers=headers)

    if verbose == True:
        print(response)
        print(type(response.text))
        print(response.text[:1000])


    return response.json()

# ƒ: Parse Reviews for final GET

In [100]:
def parse_reviews(review):
    '''Adapted from Tyrell's code'''  

    # Create empty list to store results 
#     parsed_reviews = []
    
    # Loop through each review in the list of reviews
    # Add specific k:v pairs to a dictionary      
    details = {
        'Reviewer Name': review['user']['name'],
        'Review Rating': review['rating'],
        'Review Text': review['text'],
        'Time Created': review['time_created']
        }

        # Add the new dictionary to the empty list
        
#         parsed_reviews.append(details)
       
    # Create a DataFrame from the resulting list
    
    df_parsed_reviews = pd.DataFrame.from_dict([details])
   
    return df_parsed_reviews

In [101]:
test_review = test1['reviews'][0]
test_review

{'id': '_aZ_3xAVPVJOVC_LlL0GeQ',
 'url': 'https://www.yelp.com/biz/bernardo-winery-san-diego-2?adjust_creative=eyY_X0k6PMB2bu6EM1dXdw&hrid=_aZ_3xAVPVJOVC_LlL0GeQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=eyY_X0k6PMB2bu6EM1dXdw',
 'text': 'Debbie and Ronnie were wonderfully attentive to our needs. Had the best of the both red and white wines. Well worth spending an afternoon just sipping wines...',
 'rating': 5,
 'time_created': '2021-04-13 19:18:16',
 'user': {'id': '5dlWmmwLrkLsRD_Z0697OQ',
  'profile_url': 'https://www.yelp.com/user_details?userid=5dlWmmwLrkLsRD_Z0697OQ',
  'image_url': None,
  'name': 'Patrick T.'}}

In [104]:
test2 = parse_reviews(test_review)
test2

,Reviewer Name,Review Rating,Review Text,Time Created
0,Patrick T.,5,Debbie and Ronnie were wonderfully attentive t...,2021-04-13 19:18:16


# LEFT OFF HERE

Removed stuff from Get All Reviews

In [56]:
# Process first request to Yelp API 
#     reviews = yelp_review_offset(list_of_biz_ids, yelp_key)
       
    # Parse results of first API call and save
#     parsed_reviews = parse_reviews(reviews['reviews'])
#     parsed_reviews.to_csv(file_name, index = False)
#     parsed_reviews.to_csv(file_name, mode='a', index = False)
    
    # Calculate number of pages
#     num_pages = reviews['total']//50+1
    
    # Print out confirmation feedback
#     print(f'For Business IDs: ')
#     print(f"    Total number of results: {reviews['total']}.")
#     print(f'    Total number of pages: {num_pages}.')
    
#     for biz_id in list_of_biz_ids:
#         reviews = yelp_review_offset(biz_id, yelp_key)

    # Retrieve and save remaining pages
#     for num in range(len(cleaned_ids)-1):

In [66]:
test1 = get_reviews(cleaned_ids[0], yelp_key)
print(test1.keys())

dict_keys(['reviews', 'total', 'possible_languages'])


In [67]:
print(test1['reviews'])

[{'id': '_aZ_3xAVPVJOVC_LlL0GeQ', 'url': 'https://www.yelp.com/biz/bernardo-winery-san-diego-2?adjust_creative=eyY_X0k6PMB2bu6EM1dXdw&hrid=_aZ_3xAVPVJOVC_LlL0GeQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=eyY_X0k6PMB2bu6EM1dXdw', 'text': 'Debbie and Ronnie were wonderfully attentive to our needs. Had the best of the both red and white wines. Well worth spending an afternoon just sipping wines...', 'rating': 5, 'time_created': '2021-04-13 19:18:16', 'user': {'id': '5dlWmmwLrkLsRD_Z0697OQ', 'profile_url': 'https://www.yelp.com/user_details?userid=5dlWmmwLrkLsRD_Z0697OQ', 'image_url': None, 'name': 'Patrick T.'}}, {'id': '_HFTJW4nRUe5lC8M1LLflA', 'url': 'https://www.yelp.com/biz/bernardo-winery-san-diego-2?adjust_creative=eyY_X0k6PMB2bu6EM1dXdw&hrid=_HFTJW4nRUe5lC8M1LLflA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=eyY_X0k6PMB2bu6EM1dXdw', 'text': "I haven't gone to the winery since pre covid; I used to go a few times a month for lunc

# ƒ: GET REVIEWS (ALL)

In [50]:
def get_all_reviews(list_of_biz_ids, yelp_key, file_name = 'data/reviews_raw.csv'):
    '''Requests all review results for given business IDs from Yelp API; \
    saves as a .csv; and returns a DataFrame.'''
    
    blank_df = pd.DataFrame()
    blank_df.to_csv(file_name)
        
    for i in list_of_biz_ids:
        try:
            # Process API request
            raw_reviews = get_reviews(i, yelp_key)

            # From results, take values from "Businesses" key and save
            parsed_reviews = parse_reviews(raw_reviews['reviews']) 

            print("Parsed reviews.")

            # H2: save results to df
            parsed_reviews.to_csv(file_name, mode='a', index = False)

            print("Saved reviews.")

        except:
            # If error, print where the error happens
            print(f'Error on page {num}.')
            # Then save the results so far to the .csv
            parsed_reviews.to_csv(file_name, mode='a', index = False)

        
    return parsed_reviews

#### Test GET REVIEWS

In [51]:
get_reviews(cleaned_ids, yelp_key)

TypeError: can only concatenate str (not "list") to str

In [32]:
reviews1 = pd.read_csv('data/reviews_raw.csv', header = 0)
reviews1

,Reviewer Name,Review ID,Time Created,Review Rating,Review Text
0,Patrick T.,_aZ_3xAVPVJOVC_LlL0GeQ,2021-04-13 19:18:16,5,Debbie and Ronnie were wonderfully attentive t...
1,Bianca B.,_HFTJW4nRUe5lC8M1LLflA,2021-02-21 15:29:51,5,I haven't gone to the winery since pre covid; ...
2,stephanie l.,AmyrN0N2SjWI_Uzy3Uq0jQ,2021-02-21 17:19:02,5,I can't believe I never been here before! It's...


In [33]:
reviews1

,Reviewer Name,Review ID,Time Created,Review Rating,Review Text
0,Patrick T.,_aZ_3xAVPVJOVC_LlL0GeQ,2021-04-13 19:18:16,5,Debbie and Ronnie were wonderfully attentive t...
1,Bianca B.,_HFTJW4nRUe5lC8M1LLflA,2021-02-21 15:29:51,5,I haven't gone to the winery since pre covid; ...
2,stephanie l.,AmyrN0N2SjWI_Uzy3Uq0jQ,2021-02-21 17:19:02,5,I can't believe I never been here before! It's...


## Combining Functions

- Combine the functions above into a single script  

# Part 4 -  Using python and pandas, write code to answer the questions below. 

**Reviews**

Which are the 5 most reviewed businesses in your dataset?

What is the highest rating received in your data set and how many businesses have that rating?

What percentage of businesses have a rating greater than or  4.5?

What percentage of businesses have a rating less than 3?

---

**Pricing**

What percentage of your businesses have a price label of one dollar sign? Two dollar signs? Three dollar signs? No dollar signs?

---

**Returing Reviews**

Return the text of the reviews for the most reviewed business. 

Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the business with the most reviews. 

Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the business with the least reviews. 

___

# Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [34]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 20

NameError: name 'num' is not defined